In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import time
import importlib

In [2]:
import sys
sys.path.append("../../../libs_bell")
from lib_preprocessing import job_ids_to_result, flatten_results_jobs_list, arrange_results_list_tensored3, results_list_to_hist_list

In [3]:
with open("pkls/raw_hist_list.pkl", "rb") as f:
    raw_hist_list = pickle.load(f)
with open("pkls/results_meas_cal.pkl", "rb") as f:
    results_meas_cal = pickle.load(f)

In [4]:
from libs_qrem import LeastNormFilter
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter

In [5]:
max_size = 65
max_length = 64
lnp_mitigator_list = []
for n in range(2, max_size + 1):
    mit_pattern = [[i] for i in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 1], mit_pattern=mit_pattern)
    lnp_mitigator_list.append(LeastNormFilter(n, meas_fitter.cal_matrices))
    if n % 10 == 0:
        print("size", n, "finished")
print("length of lnp_mitigator_list: ", len(lnp_mitigator_list))

size 10 finished
size 20 finished
size 30 finished
size 40 finished
size 50 finished
size 60 finished
length of lnp_mitigator_list:  64


In [6]:
for i in range(max_length):
    t1 = time.perf_counter()
    _ = lnp_mitigator_list[i].apply(raw_hist_list[i + 1])
    t2 = time.perf_counter()
    print(i + 1, "th finished (", i + 2, "qubits,", t2 - t1, "s )")

1 th finished ( 2 qubits, 8.322500000090827e-05 s )
2 th finished ( 3 qubits, 5.1782999999971935e-05 s )
3 th finished ( 4 qubits, 0.00010149399999903608 s )
4 th finished ( 5 qubits, 0.00019168299999883232 s )
5 th finished ( 6 qubits, 0.0002912169999991221 s )
6 th finished ( 7 qubits, 0.0004783599999989008 s )
7 th finished ( 8 qubits, 0.0007170789999992877 s )
8 th finished ( 9 qubits, 0.0011064810000007697 s )
9 th finished ( 10 qubits, 0.0014991860000002077 s )
10 th finished ( 11 qubits, 0.0020104740000004284 s )
11 th finished ( 12 qubits, 0.0026016260000005786 s )
12 th finished ( 13 qubits, 0.004299616000000839 s )
13 th finished ( 14 qubits, 0.008765189000000007 s )
14 th finished ( 15 qubits, 0.022455458999999678 s )
15 th finished ( 16 qubits, 0.016189802999999614 s )
16 th finished ( 17 qubits, 0.027348886000000405 s )
17 th finished ( 18 qubits, 0.033326135000001145 s )
18 th finished ( 19 qubits, 0.04665454999999952 s )
19 th finished ( 20 qubits, 0.05418881100000128 s 

In [7]:
lnp_mitigator_info = []
for i in range(max_length):
    t1 = time.perf_counter()
    lnp_mitigator_info.append({"exact_one_norm_of_reduced_inv_A": lnp_mitigator_list[i].exact_one_norm_of_reduced_inv_A(),
                                 "mitigated_hist": lnp_mitigator_list[i].mitigated_hist(),
                                 "x_s": lnp_mitigator_list[i].x_s(),
                                 "x_hat": lnp_mitigator_list[i].x_hat(),
                                 "x_tilde": lnp_mitigator_list[i].x_tilde(),
                                 "sum_of_x": lnp_mitigator_list[i].sum_of_x(),
                                 "sum_of_x_hat": lnp_mitigator_list[i].sum_of_x_hat(),
                                 "sum_of_x_tilde": lnp_mitigator_list[i].sum_of_x_tilde(),
                                 "indices_to_keys_vector": lnp_mitigator_list[i].indices_to_keys_vector(),
                                 "times": lnp_mitigator_list[i].times(),
                                 "expval": lnp_mitigator_list[i].expval(),
                                 "mitigation_stddev": lnp_mitigator_list[i].mitigation_stddev(norm_type = "exact"),
                                 })
    t2 = time.perf_counter()
    print(i + 1, "th finished (", i + 2, "qubits,", t2 - t1, "s )")

1 th finished ( 2 qubits, 0.0003635080000066182 s )
2 th finished ( 3 qubits, 9.25419999902033e-05 s )
3 th finished ( 4 qubits, 5.0324000000046e-05 s )
4 th finished ( 5 qubits, 6.053799999961029e-05 s )
5 th finished ( 6 qubits, 6.990099998915866e-05 s )
6 th finished ( 7 qubits, 9.726800000464664e-05 s )
7 th finished ( 8 qubits, 0.00011822100000813407 s )
8 th finished ( 9 qubits, 0.0010481609999999364 s )
9 th finished ( 10 qubits, 0.0017735640000040576 s )
10 th finished ( 11 qubits, 0.0009929249999913736 s )
11 th finished ( 12 qubits, 0.00022097400000120615 s )
12 th finished ( 13 qubits, 0.0008382060000116098 s )
13 th finished ( 14 qubits, 0.0003225049999997509 s )
14 th finished ( 15 qubits, 0.0003447800000060397 s )
15 th finished ( 16 qubits, 0.00242269800000372 s )
16 th finished ( 17 qubits, 0.0011131280000000743 s )
17 th finished ( 18 qubits, 0.0018205469999941215 s )
18 th finished ( 19 qubits, 0.0022618490000070324 s )
19 th finished ( 20 qubits, 0.001304339000000709

In [8]:
with open("./pkls/lnp_mitigator_info.pkl", "wb") as f:
    pickle.dump(lnp_mitigator_info, f)